In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import sys
sys.setrecursionlimit(110000)

In [4]:
booking_train = pd.read_csv('../data/account_booking_train.csv')
print(len(booking_train))
booking_test = pd.read_csv('../data/account_booking_test.csv')
print(len(booking_test))

# booking_train = pd.concat([booking_train,booking_test])
external_parties_train = pd.read_csv('../data/external_parties_train.csv')
y = external_parties_train['external_id']
external_parties_train.drop(['external_id'],axis=1)

external_parties_test = pd.read_csv('../data/external_parties_test.csv')

# external_parties_train = pd.concat([external_parties_train,external_parties_test])


11464
1511754


In [6]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('all-MiniLM-L6-v2',device='cuda')  # Model SBERT
# # try to encode unparsed column
# embeddings = model.encode(external_parties_train['party_info_unstructured'],show_progress_bar=True)
embeddings = np.load('../test_embeddings.npy')

In [7]:
from sklearn.decomposition import PCA
pca = PCA(n_components=32).fit(embeddings)
r_emb = pca.transform(embeddings)


In [29]:
sorted_ideces = external_parties_test.sort_values(['parsed_name']).index
sorted_ideces_location = external_parties_test.sort_values(['parsed_address_country','parsed_address_state','parsed_address_city','parsed_address_street_name']).index
ord_emb = torch.tensor(r_emb[sorted_ideces])
ord_location_emb = torch.tensor(r_emb[sorted_ideces_location])


In [41]:
k = 10
n = len(ord_emb)
output = torch.zeros([n-k,k,k])
indece_matrix = []
for i in range(n-k):
    x_mid = sorted_ideces[i+k//2]
    location_x_mid = sorted_ideces_location.get_loc(x_mid)
    x = torch.tensor(sorted_ideces[i:i+k])
    x_loc = torch.tensor(sorted_ideces_location[location_x_mid-k//2:location_x_mid+k//2])

    full_x = torch.cat([x,x_loc]).repeat_interleave(2*k).flatten()
    full_y =  full_x.view(2*k,2*k).T.flatten()
    # y = x.view(k,k).T.flatten()
    # y_loc = x_loc.view(k,k).T.flatten()
    indece_matrix.append(torch.stack([full_x,full_y]).T.view(2*k,2*k,-1))

    cur_norm = torch.norm(torch.stack([ord_emb[i:i+k],ord_location_emb[location_x_mid-k//2:location_x_mid+k//2]]), dim=1, keepdim=True)

    cur_emb = torch.stack([ord_emb[i:i+k],ord_location_emb[location_x_mid-k//2:location_x_mid+k//2]]) / cur_norm  # Shape (n, d)
    output[i] = torch.mm(cur_emb,cur_emb.T).triu()
    output[i].fill_diagonal_(0)

indece_matrix = torch.cat(indece_matrix).view(-1,2*k,2*k,2)

/tmp/ipykernel_269134/1429466793.py:20: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  output[i] = torch.mm(cur_emb,cur_emb.T).triu()


RuntimeError: self must be a matrix

In [11]:
# torch.save(output,'similiarity_output_72.tp')
# output_name = torch.load('similiarity_output.tp')
output_name = output

In [12]:
threshold = 0.9
merger_list = torch.unique(indece_matrix[(output_name.abs()>0.9)],dim=0)
# merger_list_location = torch.unique(indece_matrix[(output.abs()>threshold)],dim=0)
# torch.save(merger_list_location,'similiarity_output_locality.tp')

In [197]:
# full_merger_list = torch.cat([merger_list,merger_list_location])

In [14]:
class Graph:
    # init function to declare class variables
    def __init__(self, V):
        self.V = V
        self.adj = {}
        for v in V:
            self.adj[v] = []
 
    def DFSUtil(self, temp, v, visited):
 
        # Mark the current vertex as visited
        visited[v] = True
 
        # Store the vertex to list
        temp.append(v)
 
        # Repeat for all vertices adjacent
        # to this vertex v
        for i in self.adj[v]:
            if visited[i] == False:
 
                # Update the list
                temp = self.DFSUtil(temp, i, visited)
        return temp
 
    # method to add an undirected edge
    def addEdge(self, v, w):
        self.adj[v].append(w)
        self.adj[w].append(v)
 
    # Method to retrieve connected components
    # in an undirected graph
    def connectedComponents(self):
        visited = {}
        cc = []
        for i in self.V:
            visited[i] = False
        for v in self.V:
            if visited[v] == False:
                temp = []
                cc.append(self.DFSUtil(temp, v, visited))
        return cc

In [15]:
from tqdm.notebook import tqdm

g = Graph(np.arange(len(external_parties_test)))

for idx, (a,b) in tqdm(enumerate(merger_list.numpy())):
    g.addEdge(a,b)

iban_map = {}
for idx, e in external_parties_test.iterrows():
    if(not iban_map.get(e['party_iban'])):
        iban_map[iban_map.get(e['party_iban'])] = idx
    else:
        g.addEdge(idx,iban_map[e['party_iban']])


c = g.connectedComponents()

0it [00:00, ?it/s]

In [16]:
pred_id = np.arange(30000,30000+len(external_parties_test))

for idx, c_i in enumerate(c):
    for c_j in c_i:
        pred_id[c_j] = pred_id[c_i[0]]

In [22]:
np.unique(pred_id).size

1347381

In [18]:
from  test_predictions import  evaluate_datasets

dict_ = {'transaction_reference_id': external_parties_test['transaction_reference_id'], 'external_id':pred_id}

# pred_dict = pd.DataFrame.from_dict(dict_)

# evaluate_datasets(external_parties_train,pred_dict)

In [19]:
pd.DataFrame.from_dict(dict_).to_csv('../submission_5.csv',index=False)

In [3]:
import re

external_parties_train['party_number'] = (
    external_parties_train['party_phone']
    .str.replace(r'x.*$', '', regex=True)  # Removes everything after 'x' (including 'x')
    .str.replace(r'[^0-9]', '', regex=True)  # Removes all characters that are not numbers
    .str[:15]  # Keep only the first 15 numeric characters
)

def clean_and_deduplicate_country(country_name):
    if pd.isna(country_name):
        return country_name    
    # Separare le parole e rimuovere le ripetizioni
    words = country_name.split()
    unique_words = set(words)  # Usa un set per rimuovere le ripetizioni
    cleaned_country = ' '.join(sorted(unique_words))  # Ricostruire la stringa (opzionalmente ordinata)
    return cleaned_country

external_parties_train['parsed_address_country'] = external_parties_train['parsed_address_country'].str.replace(r'[().]', '', regex=True).str.replace(r'\d+', '', regex=True)
external_parties_train['parsed_address_country'] = external_parties_train['parsed_address_country'].apply(lambda x: clean_and_deduplicate_country(x))

external_parties_train['parsed_address_street_name'] = external_parties_train['parsed_address_street_name'].str.replace(r'&', '', regex=True).str.replace(r'\d+', '', regex=True)
external_parties_train[['parsed_address_street_name','parsed_address_street_name']]

external_parties_train = external_parties_train.fillna('0') #fill all nan with 0

In [4]:
from opensearchpy import OpenSearch
 
host = [{'host': 'search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.aos.us-west-2.on.aws','port':443, 'schema':'https'}]
auth = ('nemo', 'UBS@suck10')
 
from opensearchpy import OpenSearch, RequestsHttpConnection

client = OpenSearch(
    hosts = host,
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_show_warn = False,
    connection_class = RequestsHttpConnection
)

try:
    info = client.info()
    print("Connected to OpenSearch")
    print(info)
except Exception as e:
    print(f"Error connecting to OpenSearch: {e}")


index_name = "party_data"
mapping = {
    "mappings": {
        "properties": {
            "transaction_reference_id": {"type": "keyword"},
            "party_role": {"type": "text"},
            "party_info_unstructured": {"type": "text"},
            "parsed_name": {"type": "text"},
            "parsed_address_city": {"type": "text"},
            "parsed_address_country": {"type": "keyword"},
            "party_iban": {"type": "keyword"}
        }
    }
}

client.index(index_name,mapping)

Connected to OpenSearch
{'name': '490add16759d594f8c790c7a745f7861', 'cluster_name': '763504090600:ubs-open-search', 'cluster_uuid': 'TjDJUiMDQ1yo01dIuz2UwQ', 'version': {'distribution': 'opensearch', 'number': '2.17.0', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2024-11-18T04:22:32.407132088Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


{'_index': 'party_data',
 '_id': 'Flrof5MBXDht5Qx5-ya9',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 3, 'successful': 3, 'failed': 0},
 '_seq_no': 119,
 '_primary_term': 1}

In [5]:
# data_json = parties_train.to_dict(orient='records')

In [6]:
from opensearchpy import OpenSearch, helpers

def generate_data_for_opensearch(df, index_name):
    for _, row in df.iterrows():
        # print(row[['clean_parsed_address_street_name','cleaned_party_number','clean_parsed_address_country']].to_dict())
        yield {
            "_op_type": "index",  # The operation (indexing the document)
            "_index": index_name,  # Target index
            "_id": row['transaction_reference_id'],  # Optional: specify an ID
            "_source": row[['parsed_address_street_name','party_number','parsed_address_country']].to_dict()  # The document data
        }

helpers.bulk(client, generate_data_for_opensearch(external_parties_train,index_name))

(1492736, [])

In [60]:
street_name = "villerreal keys keys suite"  # Deliberate typo for fuzzy search
query = {
    "query": {
        "match": {
            "parsed_address_street_name": {
                "query": street_name,
                "minimum_should_match": "97%"  # Require all query terms to match
            }
        }
    }
}
response = client.search(index=index_name, body=query)

In [61]:
response

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1156, 'relation': 'eq'},
  'max_score': 15.099878,
  'hits': [{'_index': 'party_data',
    '_id': '42c5450e057982bbe3fc999647e99ec7',
    '_score': 15.099878,
    '_source': {'parsed_address_street_name': 'powers keys keys  suite',
     'party_number': '0',
     'parsed_address_country': '0'}},
   {'_index': 'party_data',
    '_id': '5f3fb7d8efd63125b2bfc46985c10003',
    '_score': 15.094254,
    '_source': {'parsed_address_street_name': 'proctof keys keys suite',
     'party_number': '414905628217',
     'parsed_address_country': '0'}},
   {'_index': 'party_data',
    '_id': '9c4e602278b8382c3086b17bc2addb31',
    '_score': 15.09318,
    '_source': {'parsed_address_street_name': 'villarreal keys keys suite',
     'party_number': '0',
     'parsed_address_country': 'dominica'}},
   {'_index': 'party_data',
    '_id': '29aebb7f89d3286ac74d09935c25aada',
  

In [68]:
location_sorted = external_parties_train.sort_values(['parsed_address_country','parsed_address_state','parsed_address_city','parsed_address_street_name'])

In [71]:
location_sorted[location_sorted['parsed_address_country']=='italy'].to_csv('italy.csv')

## DEAD ZONE

In [18]:
from opensearchpy import AsyncOpenSearch, AsyncHttpConnection

async def main():
    client = AsyncOpenSearch(
        hosts = [{'host': 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com', 'port': 9200}],
        http_auth = ('admin', 'admin'),
        use_ssl = True,
        verify_certs = False,
        ssl_show_warn = False,
        connection_class = AsyncHttpConnection
    )
    try:
        info = client.info().close()
        print("Connected to OpenSearch")
        print(info.)
    except Exception as e:
        print(f"Error connecting to OpenSearch: {e}")

main()

<coroutine object main at 0x71d76766d940>

In [28]:
es = Elasticsearch(
    'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com:443/',
    bearer_auth = ('nemo', 'UBS@suck10'),
    request_timeout=10,
    verify_certs=False
)
es.info()

AuthenticationException: AuthenticationException(401, 'Unauthorized')

In [30]:
from opensearchpy import OpenSearch, Urllib3HttpConnection

client = OpenSearch(
    hosts = [{'host': 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.aos.us-west-2.on.aws:443/' }],
    http_auth = ('nemo', 'UBS@suck10'),
    use_ssl = True,
    verify_certs = False,
    ssl_show_warn = False,
    connection_class = Urllib3HttpConnection
)

try:
    info = client.info()
    print("Connected to OpenSearch")
    print(info)
except Exception as e:
    print(f"Error connecting to OpenSearch: {e}")

Error connecting to OpenSearch: ConnectionError(<urllib3.connection.HTTPSConnection object at 0x71d75c77bdd0>: Failed to resolve 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.aos.us-west-2.on.aws:443/' ([Errno -2] Name or service not known)) caused by: NameResolutionError(<urllib3.connection.HTTPSConnection object at 0x71d75c77bdd0>: Failed to resolve 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.aos.us-west-2.on.aws:443/' ([Errno -2] Name or service not known))


In [13]:
from opensearchpy import OpenSearch

# Replace with your cluster details
host = "https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com"
port = 443

# Create the OpenSearch client
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    https_auth=auth,
    use_ssl=True,
    verify_certs=True,  # Set to False if SSL issues are suspected
    timeout=30
)

# Test connection
try:
    info = client.info()
    print("Connected to OpenSearch:", info)
except Exception as e:
    print("Error connecting to OpenSearch:", e)

Error connecting to OpenSearch: ConnectionError(<urllib3.connection.HTTPSConnection object at 0x79feb7f65cc0>: Failed to resolve 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com' ([Errno -2] Name or service not known)) caused by: NameResolutionError(<urllib3.connection.HTTPSConnection object at 0x79feb7f65cc0>: Failed to resolve 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com' ([Errno -2] Name or service not known))


In [14]:
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=False  # Disable for testing only
)

/home/neo/.local/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:214: UserWarning: Connecting to https://[https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com]:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [15]:
try:
    info = client.info()
    print("Connected to OpenSearch:", info)
except Exception as e:
    print("Error connecting to OpenSearch:", e)

Error connecting to OpenSearch: ConnectionError(<urllib3.connection.HTTPSConnection object at 0x79feb7f18850>: Failed to resolve 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com' ([Errno -2] Name or service not known)) caused by: NameResolutionError(<urllib3.connection.HTTPSConnection object at 0x79feb7f18850>: Failed to resolve 'https://search-ubs-open-search-qgmxboaarfs4i2p2gde7yw2xjq.us-west-2.es.amazonaws.com' ([Errno -2] Name or service not known))


In [ ]:
es = Elasticsearch()